In [1]:
import wallet

In [2]:

account1 = wallet.Account.from_phrase(b"Oscar Eduardo Serna Rosero","big")

In [3]:
account1.testnet_address

'mo3WWB4PoSHrudEBik1nUqfn1uZEPNYEc8'

In [4]:
account1

Private Key Hex: 0000000000004f73636172204564756172646f205365726e6120526f7365726f

In [5]:
account2 = wallet.Account(0x0000000000004f73636172204564756172646f205365726e6120526f7365726f)

In [6]:
account2.testnet_address

'mo3WWB4PoSHrudEBik1nUqfn1uZEPNYEc8'

In [7]:
multisig_acc = wallet.MultSigAccount.from_phrases(2,2,[(b"Oscar Eduardo Serna Rosero","big"), (b"Oscar Eduardo Serna Rosero","little")])

In [10]:
multisig_acc.testnet_address

'2NAqp3V17G4iBekEwWuYkgzoEyWiULhYZLs'

In [11]:
multisig_acc.redeem_script

OP_2 03e07f96e5ba598431c0c994493a4ae988c9854c171d5d4bb140db0a27a4c853e4 031b63f964d8c65d1d1136dcfe5033dedea88c2d411934ea48c9708410be84e5ee OP_2 OP_CHECKMULTISIG